In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import sklearn
import statistics
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import time

In [4]:
start_time = time.time()
train_data_frame = pd.read_csv('train.csv')
train_data_frame.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [5]:
train_Features = train_data_frame.iloc[:, :-1] 
# drop last column
train_Classes = train_data_frame.iloc[:, -1] 
# drop last column
RFclassifier = RandomForestClassifier()
params = RFclassifier.get_params()
print(params)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [6]:
sklearn.model_selection.cross_validate(RFclassifier, train_Features, train_Classes)


{'fit_time': array([2.20352554, 2.02581906, 2.04725099, 1.99393582, 2.02725339]),
 'score_time': array([0.06651163, 0.07496619, 0.07793379, 0.07764411, 0.08261943]),
 'test_score': array([0.44808201, 0.61507937, 0.68452381, 0.72255291, 0.73743386])}

In [7]:
search_space = {
    "max_depth": tune.grid_search([5*i for i in range(1,10)]),
    "n_estimators": tune.grid_search([10,100,200,300]),
    "ccp_alpha": tune.grid_search([0.0,0.1,0.2]),
}
ray.init(address='auto')

def train(config):
    RFclassifier = RandomForestClassifier(**config)
    params = RFclassifier.get_params()
    xval = sklearn.model_selection.cross_validate(RFclassifier, train_Features, train_Classes)
    tune.report(mean_accuracy=statistics.mean(xval["test_score"]))

analysis = tune.run(train, config=search_space)



2022-05-12 15:41:48,734	WARNING function_runner.py:598 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-05-12 15:41:51,857	INFO trial_runner.py:803 -- starting train_45cfb_00000


Trial name,status,loc,ccp_alpha,max_depth,n_estimators
train_45cfb_00000,RUNNING,192.168.2.52:39186,0,5,10
train_45cfb_00001,PENDING,,0.1,5,10
train_45cfb_00002,PENDING,,0.2,5,10
train_45cfb_00003,PENDING,,0,10,10
train_45cfb_00004,PENDING,,0.1,10,10
train_45cfb_00005,PENDING,,0.2,10,10
train_45cfb_00006,PENDING,,0,15,10
train_45cfb_00007,PENDING,,0.1,15,10
train_45cfb_00008,PENDING,,0.2,15,10
train_45cfb_00009,PENDING,,0,20,10


2022-05-12 15:41:54,892	INFO trial_runner.py:803 -- starting train_45cfb_00001
2022-05-12 15:41:55,114	INFO trial_runner.py:803 -- starting train_45cfb_00002


Result for train_45cfb_00000:
  date: 2022-05-12_15-41-55
  done: false
  experiment_id: 31ce2e1f5b7d4601bee7d2b99ac8e1cb
  hostname: patrick-c3-2
  iterations_since_restore: 1
  mean_accuracy: 0.5873677248677248
  node_ip: 192.168.2.52
  pid: 39186
  time_since_restore: 0.4887809753417969
  time_this_iter_s: 0.4887809753417969
  time_total_s: 0.4887809753417969
  timestamp: 1652362915
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 45cfb_00000
  warmup_time: 0.010429859161376953
  
Result for train_45cfb_00000:
  date: 2022-05-12_15-41-55
  done: true
  experiment_id: 31ce2e1f5b7d4601bee7d2b99ac8e1cb
  experiment_tag: 0_ccp_alpha=0.0,max_depth=5,n_estimators=10
  hostname: patrick-c3-2
  iterations_since_restore: 1
  mean_accuracy: 0.5873677248677248
  node_ip: 192.168.2.52
  pid: 39186
  time_since_restore: 0.4887809753417969
  time_this_iter_s: 0.4887809753417969
  time_total_s: 0.4887809753417969
  timestamp: 1652362915
  timesteps_since_restore: 0
  training_itera

2022-05-12 15:41:57,010	INFO trial_runner.py:803 -- starting train_45cfb_00003


...

Result for train_45cfb_00106:
  date: 2022-05-12_15-56-52
  done: false
  experiment_id: 4f4f1639f0ea45fcb477019c22b92315
  hostname: patrick-c3-2
  iterations_since_restore: 1
  mean_accuracy: 0.31977513227513227
  node_ip: 192.168.2.52
  pid: 44209
  time_since_restore: 54.50960659980774
  time_this_iter_s: 54.50960659980774
  time_total_s: 54.50960659980774
  timestamp: 1652363812
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 45cfb_00106
  warmup_time: 0.005877256393432617
  
Result for train_45cfb_00106:
  date: 2022-05-12_15-56-52
  done: true
  experiment_id: 4f4f1639f0ea45fcb477019c22b92315
  experiment_tag: 106_ccp_alpha=0.1,max_depth=45,n_estimators=300
  hostname: patrick-c3-2
  iterations_since_restore: 1
  mean_accuracy: 0.31977513227513227
  node_ip: 192.168.2.52
  pid: 44209
  time_since_restore: 54.50960659980774
  time_this_iter_s: 54.50960659980774
  time_total_s: 54.50960659980774
  timestamp: 1652363812
  timesteps_since_restore: 0
  training_itera

Trial name,status,loc,ccp_alpha,max_depth,n_estimators,acc,iter,total time (s)
train_45cfb_00107,RUNNING,192.168.2.133:6863,0.2,45,300,0.142857,1,54.3451
train_45cfb_00000,TERMINATED,192.168.2.52:39186,0,5,10,0.587368,1,0.488781
train_45cfb_00001,TERMINATED,192.168.2.133:2710,0.1,5,10,0.289352,1,0.586137
train_45cfb_00002,TERMINATED,192.168.2.133:2713,0.2,5,10,0.142857,1,0.538489
train_45cfb_00003,TERMINATED,192.168.2.52:39326,0,10,10,0.616138,1,0.753644
train_45cfb_00004,TERMINATED,192.168.2.52:39465,0.1,10,10,0.287765,1,0.786424
train_45cfb_00005,TERMINATED,192.168.2.133:2869,0.2,10,10,0.142857,1,0.915145
train_45cfb_00006,TERMINATED,192.168.2.133:2872,0,15,10,0.628175,1,1.07134
train_45cfb_00007,TERMINATED,192.168.2.52:39605,0.1,15,10,0.298611,1,1.2017
train_45cfb_00008,TERMINATED,192.168.2.133:3003,0.2,15,10,0.142857,1,1.32892


Result for train_45cfb_00107:
  date: 2022-05-12_15-56-54
  done: true
  experiment_id: 4ff1b658ea964aadb90c8d150c8d327f
  experiment_tag: 107_ccp_alpha=0.2,max_depth=45,n_estimators=300
  hostname: patrick-c3-1
  iterations_since_restore: 1
  mean_accuracy: 0.14285714285714285
  node_ip: 192.168.2.133
  pid: 6863
  time_since_restore: 54.34512424468994
  time_this_iter_s: 54.34512424468994
  time_total_s: 54.34512424468994
  timestamp: 1652363814
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 45cfb_00107
  warmup_time: 0.005118131637573242
  


Trial name,status,loc,ccp_alpha,max_depth,n_estimators,acc,iter,total time (s)
train_45cfb_00000,TERMINATED,192.168.2.52:39186,0,5,10,0.587368,1,0.488781
train_45cfb_00001,TERMINATED,192.168.2.133:2710,0.1,5,10,0.289352,1,0.586137
train_45cfb_00002,TERMINATED,192.168.2.133:2713,0.2,5,10,0.142857,1,0.538489
train_45cfb_00003,TERMINATED,192.168.2.52:39326,0,10,10,0.616138,1,0.753644
train_45cfb_00004,TERMINATED,192.168.2.52:39465,0.1,10,10,0.287765,1,0.786424
train_45cfb_00005,TERMINATED,192.168.2.133:2869,0.2,10,10,0.142857,1,0.915145
train_45cfb_00006,TERMINATED,192.168.2.133:2872,0,15,10,0.628175,1,1.07134
train_45cfb_00007,TERMINATED,192.168.2.52:39605,0.1,15,10,0.298611,1,1.2017
train_45cfb_00008,TERMINATED,192.168.2.133:3003,0.2,15,10,0.142857,1,1.32892
train_45cfb_00009,TERMINATED,192.168.2.52:39744,0,20,10,0.619048,1,1.11504


2022-05-12 15:56:54,932	INFO tune.py:701 -- Total run time: 906.21 seconds (905.62 seconds for the tuning loop).


In [8]:
stop_time = time.time()
print("Stopping at :", stop_time)
print("Total elapsed time: ", stop_time - start_time)

Stopping at : 1652363815.6093857
Total elapsed time:  918.6675434112549


In [9]:
ray.shutdown()